In [100]:
import json, time, re
import pandas as pd
from enum import Enum

In [91]:
str(LogLevel(LogLevel.Info.value))

'LogLevel.Info'

In [180]:
class LogLevel(Enum):
     Info = 10
     Debug = 20
     Warn = 30
     Exception = 50

class Logger:
    __log = []
    Verbose = False
    def __init__(self, Level): 
        self.logLevel = Level 
        self.__log = []
        self.time_init = time.time()
        self.Verbose = False
    def __get_log_item(self, loglevel, src, msg):
        d = {}
        d['level'] = str(LogLevel(loglevel))
        d['severity'] = loglevel.value
        d['time'] = self.__log_time()
        d['source'] = src
        d['message'] = msg
        return d
    def __log_time(self):
        return str(round(time.time()-self.time_init, 2))
    def LogInfo(self, source, message): 
        self.__log.append(self.__get_log_item(LogLevel.Info, source, message))  
    def Debug(self, source, message):
        d = self.__get_log_item(LogLevel.Debug, source, message)
        self.__log.append(d) 
        if self.Verbose: 
            print(f" ({d['time']}) {d['source']}: {d['message']} ")
    def LogWarn(self, source, message):
        self.__log.append(self.__get_log_item(LogLevel.Warn, source, message)) 
    def LogException(self, source, message):
        self.__log.append(self.__get_log_item(LogLevel.Exception, source, message)) 
    def get_log(self):
        logs = [ l for l in self.__log if l['severity'] >= self.logLevel.value ]
        return logs  
    def show(self): 
        for d in self.get_log():
            print(f" {LogLevel(d['severity'])}: {d['source']} {d['message']} ({d['time']}) " )
    def to_json(self, path):  
        with open(f'{path}', "w") as f:
            json.dump(self.get_log(), f, indent=4 )
 

In [182]:
logger = Logger(LogLevel.Info)  
logger.Verbose = True
logger.LogInfo('main','trying')

try: 
    logger.LogInfo('main','open connection')
    logger.LogWarn('main','warning') 
    time.sleep(.20)
    err  
except Exception as e:
    logger.LogException('main',f'Exception {e}')          
finally: 
    logger.LogInfo('main','close connection')     
    
time.sleep(.20) 
logger.Debug('main', f'block complete'  )
 
logger.to_json(r'C:\\temp\\log.json')
df = pd.DataFrame(logger.get_log() )
df


 (0.42) main: block complete 


,level,severity,time,source,message
0,LogLevel.Info,10,0.0,main,trying
1,LogLevel.Info,10,0.0,main,open connection
2,LogLevel.Warn,30,0.0,main,warning
3,LogLevel.Exception,50,0.22,main,Exception name 'err' is not defined
4,LogLevel.Info,10,0.22,main,close connection
5,LogLevel.Debug,20,0.42,main,block complete
